## 2 room environment

In [ ]:
class Environment2Room:
    def __init__(self):
        # Initial state of the environment: a list of two rooms
        # The left room contains both the agent and dirt ("AD"), and the right room contains only dirt ("D")
        # "A" represents the agent, "D" represents dirt, and an empty string represents a clean room with no agent
        self.state = ["AD", "D"]  # Agent starts in the left room, both rooms are dirty

    def __str__(self):
        # Provides a string representation of the current state of the environment
        return f"Environment state: {self.state}"

    def getPerceptFromEnvironment(self):
        # Generates a percept for the agent to act upon
        # The percept is a dictionary containing the agent's location and whether the current room is dirty

        percept = {}  # Initialize an empty dictionary to store percepts

        # Check if the agent is in the left room
        if "A" in self.state[0]:
            percept["agent_location"] = "L"  # Agent is in the left room
            percept["dirty"] = "D" in self.state[0]  # Check if the left room is dirty
        # Check if the agent is in the right room
        elif "A" in self.state[1]:
            percept["agent_location"] = "R"  # Agent is in the right room
            percept["dirty"] = "D" in self.state[1]  # Check if the right room is dirty

        return percept  # Return the percept for the agent to use

    def setEnvironment(self, action):
        # This method updates the environment's state based on the agent's action
        # It supports cleaning the current room or moving between rooms

        # Determine which room the agent is in (left or right)
        agent_location = "L" if "A" in self.state[0] else "R"

        if action == "clean":
            # If the action is "clean", remove the dirt from the current room
            if agent_location == "L":
                self.state[0] = self.state[0].replace("D", "")  # Clean the left room
            else:
                self.state[1] = self.state[1].replace("D", "")  # Clean the right room

        elif action == "moveR" and agent_location == "L":
            # If the action is "moveR", move the agent from the left room to the right room
            self.state[0] = self.state[0].replace("A", "")  # Remove agent from the left room
            self.state[1] = "A" + self.state[1]  # Add agent to the right room

        elif action == "moveL" and agent_location == "R":
            # If the action is "moveL", move the agent from the right room to the left room
            self.state[1] = self.state[1].replace("A", "")  # Remove agent from the right room
            self.state[0] = "A" + self.state[0]  # Add agent to the left room


class ReflexAgent2Room:
    def __init__(self):
        # Reflex agent initialization
        # Reflex agents act solely based on the current percept and do not store internal state
        pass

    def __str__(self):
        # Provides a string representation of the reflex agent
        return "Reflex Agent"

    def selectAction(self, percept):
        """
        YOUR CODE GOES HERE
        """
        pass

    def execAction(self, action, environment):
        # Executes the chosen action by modifying the environment's state
        environment.setEnvironment(action)

    def perceiveAndAct(self, environment):
        # This method manages the process of perception and action execution
        # 1. Get the current percept from the environment
        percept = environment.getPerceptFromEnvironment()
        print(f"Perceived: {percept}")  # Output the percept for clarity

        # 2. Select an action based on the percept
        action = self.selectAction(percept)
        print(f"Action selected: {action}")  # Output the selected action

        # 3. Execute the selected action and update the environment's state
        self.execAction(action, environment)
        print(f"New Environment state: {environment}")  # Output the updated environment state


### Check solution

In [ ]:
def run_simulation_2_room():
    # Initialize the environment and the agent
    # The environment is a 2-room setup where both rooms are initially dirty
    # The agent is a reflex agent that makes decisions based on its current percept
    environment = Environment2Room()
    agent = ReflexAgent2Room()

    # Initialize a step counter to track how many actions the agent takes
    steps = 0
    print("Initial state:", environment.state)  # Display the initial state of the environment

    # Run the simulation loop until both rooms are clean
    # The loop continues as long as there is dirt ("D") in either room
    while "D" in environment.state[0] or "D" in environment.state[1]:
        print(f"Step {steps}:")  # Display the current step number

        # The agent perceives the current environment and performs an action
        agent.perceiveAndAct(environment)

        # Increment the step counter after each action
        steps += 1

    # Once the loop ends, the environment is clean
    print("The environment is now clean!")  # Output a message indicating the simulation is complete

# Run the simulation by calling the function
run_simulation_2_room()


Initial state: ['AD', 'D']
Step 0:
Perceived: {'agent_location': 'L', 'dirty': True}
Action selected: clean
New Environment state: Environment state: ['A', 'D']
Step 1:
Perceived: {'agent_location': 'L', 'dirty': False}
Action selected: moveR
New Environment state: Environment state: ['', 'AD']
Step 2:
Perceived: {'agent_location': 'R', 'dirty': True}
Action selected: clean
New Environment state: Environment state: ['', 'A']
The environment is now clean!


## N room environment

In [ ]:
import random

class NRoomEnvironment:
    def __init__(self, n):
        # Initialize an environment with 'n' rooms. All rooms are initially dirty.
        # Place the agent in a random room. The agent's position is chosen randomly from the available rooms.
        agent_position = random.randint(0, n - 1)  # Randomly select a room for the agent
        self.state = ["D"] * n  # Initialize all rooms as dirty (represented by "D")
        self.state[agent_position] = "AD"  # Place agent in the randomly selected room and mark it dirty
        self.n = n  # Store the number of rooms in the environment

    def __str__(self):
        # Provides a string representation of the current environment state
        return f"Environment state: {self.state}"

    def getPerceptFromEnvironment(self):
        # Generate the agent's percept from the current state of the environment
        percept = {}

        # Identify the agent's location by finding which room contains "A"
        agent_location = next(i for i, room in enumerate(self.state) if "A" in room)

        percept["dirty"] = "D" in self.state[agent_location]  # Check if the current room is dirty

        # Check if the agent is in the leftmost or rightmost room (adjacent to a wall)
        percept["wall"] = (agent_location == 0 or agent_location == self.n - 1)

        return percept  # Return the percept for the agent to use

    def setEnvironment(self, action):
        # Update the environment's state based on the agent's chosen action

        # Find the current location of the agent
        agent_location = next(i for i, room in enumerate(self.state) if "A" in room)

        if action == "clean":
            # Clean the current room (remove "D" from the room's state)
            self.state[agent_location] = self.state[agent_location].replace("D", "")

        elif action == "moveR" and agent_location < self.n - 1:
            # Move the agent to the right if not in the rightmost room
            self.state[agent_location] = self.state[agent_location].replace("A", "")  # Remove agent from the current room
            self.state[agent_location + 1] = "A" + self.state[agent_location + 1]  # Place agent in the next room to the right

        elif action == "moveL" and agent_location > 0:
            # Move the agent to the left if not in the leftmost room
            self.state[agent_location] = self.state[agent_location].replace("A", "")  # Remove agent from the current room
            self.state[agent_location - 1] = "A" + self.state[agent_location - 1]  # Place agent in the next room to the left


class MemoryAgentNRooms:
    def __init__(self):
        # Initialize the memory to store the agent's last two actions
        self.memory = []  # Memory is initially empty

    def __str__(self):
        # Provides a string representation of the memory-based agent
        return "Memory-based Agent"

    def selectAction(self, percept):
        """
        YOUR CODE GOES HERE
        """
        pass

    def execAction(self, action, environment):
        # Executes the chosen action in the environment and updates the agent's memory

        # Update the environment's state based on the action
        environment.setEnvironment(action)

        # Add the action to the agent's memory (store up to the last two actions)
        self.memory.append(action)
        if len(self.memory) > 2:
            self.memory.pop(0)  # Keep only the last two actions in memory

    def perceiveAndAct(self, environment):
        # This method handles the perception and action execution process

        # 1. Get the current percept from the environment
        percept = environment.getPerceptFromEnvironment()
        print(f"Perceived: {percept}")  # Output the percept for clarity

        # 2. Select an action based on the percept and the agent's memory
        action = self.selectAction(percept)
        print(f"Action selected: {action}")  # Output the selected action

        # 3. Execute the selected action and update the environment
        self.execAction(action, environment)
        print(f"New Environment state: {environment}")  # Output the updated environment state


## Test the solution

In [ ]:
# Function to run the simulation
def run_simulation_n_rooms(n):
    # Initialize the environment and the agent
    # The environment is an n-room setup, where all rooms start dirty, and the agent is placed randomly in one of them
    environment = NRoomEnvironment(n)
    agent = MemoryAgentNRooms()  # The agent uses memory to decide its actions based on previous steps

    # Initialize a step counter to track how many actions the agent takes
    steps = 0
    print("Initial state:", environment.state)  # Display the initial state of the environment

    # Run the simulation until all rooms are clean
    # The loop continues as long as any room contains dirt ("D")
    while any("D" in room for room in environment.state):
        print(f"Step {steps}:")  # Display the current step number

        # The agent perceives the current environment state and performs an action
        agent.perceiveAndAct(environment)

        # Increment the step counter after each action
        steps += 1

    # Once the loop ends, all rooms are clean
    print("The environment is now clean!")  # Output a message indicating the simulation is complete
    print(f"Total steps: {steps}")  # Output the total number of steps taken to clean all rooms
    print("================================================")  # Separator for clarity when running multiple tests


# Test the solution with 'n' rooms (e.g., n=8)
# This loop runs the simulation 50 times for an environment with 8 rooms to test consistency
for i in range(50):
    run_simulation_n_rooms(8)  # Run the simulation for an 8-room environment


Initial state: ['D', 'D', 'D', 'D', 'AD', 'D', 'D', 'D']
Step 0:
Perceived: {'agent_location': 4, 'dirty': True, 'wall': False}
Action selected: clean
New Environment state: Environment state: ['D', 'D', 'D', 'D', 'A', 'D', 'D', 'D']
Step 1:
Perceived: {'agent_location': 4, 'dirty': False, 'wall': False}
Action selected: moveR
New Environment state: Environment state: ['D', 'D', 'D', 'D', '', 'AD', 'D', 'D']
Step 2:
Perceived: {'agent_location': 5, 'dirty': True, 'wall': False}
Action selected: clean
New Environment state: Environment state: ['D', 'D', 'D', 'D', '', 'A', 'D', 'D']
Step 3:
Perceived: {'agent_location': 5, 'dirty': False, 'wall': False}
Action selected: moveR
New Environment state: Environment state: ['D', 'D', 'D', 'D', '', '', 'AD', 'D']
Step 4:
Perceived: {'agent_location': 6, 'dirty': True, 'wall': False}
Action selected: clean
New Environment state: Environment state: ['D', 'D', 'D', 'D', '', '', 'A', 'D']
Step 5:
Perceived: {'agent_location': 6, 'dirty': False, 'wa












































## NxM environment

In [ ]:
class NxMRoomEnvironment:
    def __init__(self, n, m):
        # Initialize an n x m grid environment where all rooms are initially dirty.
        self.n = n  # Number of rows
        self.m = m  # Number of columns

        # Create a grid where each room is marked as dirty (represented by "D").
        self.state = [["D" for _ in range(m)] for _ in range(n)]

        # Randomly place the agent in one of the rooms, ensuring the agent is placed within the grid.
        self.agent_position = [random.randint(1, n-1), random.randint(1, m-1)]  # Random position for agent
        self.state[self.agent_position[0]][self.agent_position[1]] = "AD"  # Place agent in the randomly selected room

    def __str__(self):
        # Provide a human-readable string representation of the environment grid.
        grid = "\n".join([" ".join(row) for row in self.state])  # Format the grid into rows for printing
        return f"Environment state:\n{grid}"  # Return the formatted grid

    def getPerceptFromEnvironment(self):
        # Generate the agent's percept based on its current state in the environment.
        percept = {}

        # Identify the agent's current location in the grid.
        x, y = self.agent_position

        percept["dirty"] = "D" in self.state[x][y]  # Check if the current room is dirty

        # Check if the agent is adjacent to any walls (left, right, upper, or lower boundary).
        walls = []
        if y == 0:
            walls.append("L")  # Left wall
        if y == self.m - 1:
            walls.append("R")  # Right wall
        if x == 0:
            walls.append("U")  # Upper wall
        if x == self.n - 1:
            walls.append("D")  # Bottom wall

        percept["walls"] = walls  # Store the walls percept (if any)

        return percept  # Return the percept for the agent to use

    def setEnvironment(self, action):
        # Update the environment based on the agent's chosen action.

        # Find the current location of the agent.
        x, y = self.agent_position

        if action == "clean":
            # Clean the current room by removing "D" (dirt).
            self.state[x][y] = self.state[x][y].replace("D", "")

        elif action == "moveR" and y < self.m - 1:
            # Move the agent one room to the right, if not at the rightmost edge.
            self.state[x][y] = self.state[x][y].replace("A", "")  # Remove agent from current room
            self.agent_position = [x, y + 1]  # Update agent position
            self.state[x][y + 1] = "A" + self.state[x][y + 1]  # Add agent to the new room

        elif action == "moveL" and y > 0:
            # Move the agent one room to the left, if not at the leftmost edge.
            self.state[x][y] = self.state[x][y].replace("A", "")  # Remove agent from current room
            self.agent_position = [x, y - 1]  # Update agent position
            self.state[x][y - 1] = "A" + self.state[x][y - 1]  # Add agent to the new room

        elif action == "moveU" and x > 0:
            # Move the agent one room up, if not at the topmost edge.
            self.state[x][y] = self.state[x][y].replace("A", "")  # Remove agent from current room
            self.agent_position = [x - 1, y]  # Update agent position
            self.state[x - 1][y] = "A" + self.state[x - 1][y]  # Add agent to the new room

        elif action == "moveD" and x < self.n - 1:
            # Move the agent one room down, if not at the bottommost edge.
            self.state[x][y] = self.state[x][y].replace("A", "")  # Remove agent from current room
            self.agent_position = [x + 1, y]  # Update agent position
            self.state[x + 1][y] = "A" + self.state[x + 1][y]  # Add agent to the new room


class MemoryAgentNXNRooms:
    def __init__(self):
        # Initialize a memory-based agent that stores its last two actions.
        self.memory = []  # Memory starts empty

    def __str__(self):
        # Provide a string representation of the memory-based agent.
        return "Memory-based Agent"

    def selectAction(self, percept):
        """
        YOUR CODE GOES HERE
        """
        pass

    def execAction(self, action, environment):
        # Executes the agent's action and updates the environment accordingly.
        environment.setEnvironment(action)  # Update the environment based on the chosen action

        # Add the action to the agent's memory, maintaining only the last two actions.
        self.memory.append(action)
        if len(self.memory) > 2:
            self.memory.pop(0)  # Limit memory to the last two actions

    def perceiveAndAct(self, environment):
        # Handle the process of perception and action execution.

        # 1. Get the current percept from the environment.
        percept = environment.getPerceptFromEnvironment()
        print(f"Perceived: {percept}")  # Output the percept for clarity

        # 2. Select an action based on the percept and the agent's memory.
        action = self.selectAction(percept)
        print(f"Action selected: {action}")  # Output the selected action

        # 3. Execute the selected action and update the environment.
        self.execAction(action, environment)
        print(f"New Environment state: {environment}")  # Output the updated environment state


## Test the solution

In [ ]:
# Function to run the simulation for an n x m grid of rooms
def run_simulation_nxm_rooms(n, m):
    # Initialize the environment and the agent
    # The environment is an n x m grid where each room is initially dirty, and the agent is placed in a random room
    environment = NxMRoomEnvironment(n, m)
    agent = MemoryAgentNXNRooms()  # The agent uses memory to track its last actions

    # Initialize a step counter to track how many actions the agent takes
    steps = 0
    print("Initial state:", environment.state)  # Display the initial state of the environment

    # Run the simulation until all rooms in the grid are clean
    # The loop continues as long as any room in the environment still contains dirt ("D")
    while any("D" in room for row in environment.state for room in row):
        print(f"Step {steps}:")  # Display the current step number

        # The agent perceives the environment and performs an action
        agent.perceiveAndAct(environment)

        # Increment the step counter after each action
        steps += 1

    # Once all rooms are clean, the loop ends
    print("The environment is now clean!")  # Output a message indicating the environment is fully cleaned
    print(f"Total steps: {steps}")  # Output the total number of steps taken by the agent
    print(f"==================================================")  # Separator for clarity between different test runs

# Test the solution with an n x m room environment (e.g., n=5, m=6)
# This loop runs the simulation multiple times to test behavior in different random configurations
for i in range(220):
    run_simulation_nxm_rooms(5, 6)  # Run the simulation for a 5x6 grid environment


## Solución de apartados i y ii del problema I


Describe tu solución aquí.
